# Recurrent Neural Network

In this exercise, we'll implement a 'vanilla' recurrent neural network and apply it to a character-level language modeling task.

In [1]:
# Setup and load data
import numpy as np

data = file('data/input.txt', 'r').read()
chars = list(set(data))

char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

In [2]:
class RNN:
    def __init__(self, input_size = 100, hidden_size = 100):
        self.input_size = input_size
        self.hidden_size = hidden_size

        self.W_xh = np.random.randn(self.hidden_size, self.input_size) * 0.01
        self.W_hh = np.random.randn(self.hidden_size, self.hidden_size) * 0.01
        self.W_hy = np.random.randn(self.input_size, self.hidden_size) * 0.01
        self.b_hh = np.zeros((self.hidden_size, 1))
        self.b_hy = np.zeros((self.input_size, 1))

## Forward Pass

Let's implement the forward pass and cross-entropy loss calculation given by the following equations.

$$ \mathbf{h_t} = \tanh({W_{xh}\mathbf{x_t} + W_{hh}\mathbf{h_{t-1}}}) $$

$$ \mathbf{p_t} = softmax(W_{hy}\mathbf{h_t}) $$

$$ L = - \sum y_n\log(p_n) $$

Here, $ \mathbf{x_t} $ is the 1-of-k encoded input at time $ t $, $ \mathbf{h_t} $ is the hidden state at time $ t $ and $ \mathbf{p_t} $ is the output character probabilities at time $ t $.

In [3]:
def forward(self, inputs, targets, hprev):
    xs = {} # 1-of-k encoded input
    hs = {} # Hidden state
    hs[-1] = np.copy(hprev)
    ys = {} # Unnormalized log probabilities for next characters
    ps = {} # Probabilities of next characters
    loss = 0
    
    # Iterate through every character in input, and
    # calculate xs, hs, ys, ps and cross-entropy loss
    # TODO
    #
    # END OF YOUR CODE
    
    return xs, hs, ps, loss

RNN.forward = forward

To check if our implementation is correct, we can compare the actual loss with expected loss for random predictions. If we initialize with equal weights, actual and expected loss would be exactly equal.

In [4]:
seq_length = 25

inputs = [char_to_ix[ch] for ch in data[0:seq_length]]
targets = [char_to_ix[ch] for ch in data[1:seq_length+1]]

r = RNN(len(chars))

loss_expected = -np.log(1.0/r.input_size) * seq_length
_, _, _, loss_actual = r.forward(inputs, targets, np.zeros((r.hidden_size, 1)))

print "Expected loss for random predictions: %f" % loss_expected
print "Actual loss: %f" % loss_actual

Expected loss for random predictions: 115.378013
Actual loss: 115.379292


## Backward Pass

Next, we implement the backward pass. This is just application of the chain rule. One thing to keep in mind is that since RNN parameters are shared across time, gradient at each output depends not only on the current time step, but also the previous time steps.

In [5]:
def backward(self, input, targets, xs, hs, ps):
    gW_xh, gW_hh, gW_hy = np.zeros_like(self.W_xh), np.zeros_like(self.W_hh), np.zeros_like(self.W_hy)
    gb_hh, gb_hy = np.zeros_like(self.b_hh), np.zeros_like(self.b_hy)
    dhnext = np.zeros_like(hs[0])

    # Iterate through every character in reversed input, and
    # write backpropagation step
    # TODO
    #
    # END OF YOUR CODE
    
    # Clipping gradients
    for gparam in [gW_xh, gW_hh, gW_hy, gb_hh, gb_hy]:
        np.clip(gparam, -5, 5, out = gparam)

    return gW_xh, gW_hh, gW_hy, gb_hh, gb_hy

RNN.backward = backward

It's always a good idea to compare the analytic and numerical gradients to check if our backpropagation is implemented correctly.

In [15]:
seq_length = 25

inputs = [char_to_ix[ch] for ch in data[0:seq_length]]
targets = [char_to_ix[ch] for ch in data[1:seq_length+1]]

r = RNN(len(chars))

delta = 1e-5

xs, hs, ps, loss_1 = r.forward(inputs, targets, np.zeros((r.hidden_size, 1)))
gW_xh, gW_hh, gW_hy, gb_hh, gb_hy = r.backward(inputs, targets, xs, hs, ps)
hprev = hs[len(inputs)-1]

for param, gparam in zip([r.W_xh, r.W_hh, r.W_hy, r.b_hh, r.b_hy],
                         [gW_xh, gW_hh, gW_hy, gb_hh, gb_hy]):
    ix = np.random.randint(0, param.size)
    old_val = param.flat[ix]
    
    param.flat[ix] = old_val + delta
    xs, hs, ps, loss_1 = r.forward(inputs, targets, hprev)
    _, _, _, _, _ = r.backward(inputs, targets, xs, hs, ps)
    
    param.flat[ix] = old_val - delta
    xs, hs, ps, loss_2 = r.forward(inputs, targets, hprev)
    _, _, _, _, _ = r.backward(inputs, targets, xs, hs, ps)
    
    param.flat[ix] = old_val
    
    grad_analytic = gparam.flat[ix]
    grad_numerical = (loss_1 - loss_2) / (2 * delta)
    rel_error = abs(grad_analytic - grad_numerical) / abs(grad_numerical + grad_analytic)
    print '%f, %f => %e ' % (grad_numerical, grad_analytic, rel_error)

-0.006931, -0.006931 => 2.454077e-09 
-0.000545, -0.000309 => 2.763630e-01 
0.000704, 0.000710 => 3.867996e-03 
0.005532, 0.005532 => 1.387140e-05 
0.247394, 0.247392 => 3.887931e-06 


## Sampling and Beam search

Given the distribution of character probabilities at each time step, there are several methods to obtain a single character that gets fed back into the RNN as input at the next time step.

1. Taking the maximum at the current time step.
2. Sampling from the distribution given by the softmax.
3. Taking the top k and implementing beam search.

Here, we provide a completed implementation of (2) to sample from the distribution. Fill in the missing code to implement beam search (3).

Beam Search is a search algorithm similar to Breadth First Search (BFS). More details can be found [here](http://www.cs.cmu.edu/afs/cs.cmu.edu/academic/class/46927-f97/slides/Lec3/sld023.htm) and [here](https://www.youtube.com/watch?v=G_teUutyC3Y)

In [16]:
def sample(self, seed_ix, n, hprev):
    h = np.copy(hprev)
    x = np.zeros((self.input_size, 1))
    x[seed_ix] = 1
    ixes = []
    for i in xrange(n):
        h = np.tanh(np.dot(self.W_xh, x) + np.dot(self.W_hh, h) + self.b_hh)
        y = np.dot(self.W_hy, h) + self.b_hy
        p = np.exp(y) / np.sum(np.exp(y))
        ix = np.random.choice(range(self.input_size), p=p.ravel())
        x = np.zeros((self.input_size, 1))
        x[ix] = 1
        ixes.append(ix)
    return ixes

RNN.sample = sample

In [17]:
def beam_search(self, seed_ix, n, hprev, beam_size = 5):
    h = np.copy(hprev)
    beams = [(0.0, [], h)] # log probabilities, indices, hidden state
    while n:
        # Select the top k characters from the distribution,
        # and maintain a buffer of the respective hidden states
        # TODO
        # beam_candidates = []
        # for b in beams:
        #
        # beams = beam_candidates[:beam_size]
        # END OF YOUR CODE
        n -= 1
    samples = [b[1] for b in beams]
    return samples

RNN.beam_search = beam_search

## Finally, we put them together!

### Data

Input: A sequence of characters (indices correspoding to the characters in the vocabulary) sampled from the input text file of length `seq_length`

Target: Input sequence shifted by 1 and sampled from input text ie. if input is `x[t]`, then target is `x[t+1]` where `t` is iterating over the characters in the input text file. 

The task is set as a classification task where given the index of the current character, the model learns to predict the index of the next character from the vocabulary. 

### Model 

As we are using a 1-hot representation of the input, the input size of the RNN corresponds to the size of the vocabulary. Increasing the hidden size increases the number of parameters to learn. 

### Training

In the *train* method. We use the [**AdaGrad**](http://cs231n.github.io/neural-networks-3/#ada) update to tweak our parameters. Typically, [perplexity](https://web.stanford.edu/class/cs124/lec/languagemodeling.pdf) is used as a measure the capability of the model to correctly predict the next sample. In this version only the softmax loss is calculated and used to decide the 'better' model. 

In [18]:
def train(self, data, learning_rate = 1e-1, seq_length = 25):
    n = 0
    p = 0
    mW_xh, mW_hh, mW_hy = np.zeros_like(self.W_xh), np.zeros_like(self.W_hh), np.zeros_like(self.W_hy)
    mb_hh, mb_hy = np.zeros_like(self.b_hh), np.zeros_like(self.b_hy)
    smooth_loss = -np.log(1.0/self.input_size)*seq_length
    while True:
        if p + seq_length >= len(data) or n == 0:
            hprev = np.zeros((self.hidden_size, 1))
            p = 0

        inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
        targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

        if n % 100 == 0:
            samples = self.beam_search(inputs[0], seq_length, hprev, 5)
            for sample_ix in samples:
                txt = ''.join(ix_to_char[ix] for ix in sample_ix)
                print '----\n %s \n----' % (txt)

        xs, hs, ps, loss = self.forward(inputs, targets, hprev)
        gW_xh, gW_hh, gW_hy, gb_hh, gb_hy = self.backward(inputs, targets, xs, hs, ps)

        hprev = hs[len(inputs)-1]

        smooth_loss = smooth_loss * 0.999 + loss * 0.001
        if n % 100 == 0: print 'iter %d, loss: %f' % (n, smooth_loss)

        for param, gparam, mem in zip([self.W_xh, self.W_hh, self.W_hy, self.b_hh, self.b_hy],
                            [gW_xh, gW_hh, gW_hy, gb_hh, gb_hy],
                            [mW_xh, mW_hh, mW_hy, mb_hh, mb_hy]):
            mem += gparam * gparam
            param += -learning_rate * gparam / np.sqrt(mem + 1e-8) # adagrad update

        p += seq_length
        n += 1

RNN.train = train

In [ ]:
r = RNN(len(chars), 200)
r.train(data, learning_rate = 1e-2, seq_length = 25)